### Notebook explotar la información electoral de las elecciones generales al Congreso de los Diputados


Este notebook ha sido creado para obtener archivos geoespaciales (geojson y/ shapefile) con la información de los resultados electorales a nivel de sección censal.

Desde el Ministerio del Interior de España se pone a disposición de los ciudadanos la información relativa a las elecciones en 3 niveles: mesa electoral, municipio y superior a municipio.

En el contexto de las elecciones en España, las mesas electorales forman parte de la Administración Electoral, junto con las Juntas Electorales. A la Mesa electoral le corresponde presidir el acto de la votación, controlar el desarrollo de la votación y realizar el recuento y el escrutinio.

En términos geográficos cada sección censal* esta compuesta por al menos una mesa electoral (esto dependerá del volumen de personas adscritas a dicha sección censal), por lo que para explotar los datos con el objetivo de poder representar los resultados geográficamente debemos agregar los resultados de las mesas electorales hasta el nivel de sección censal.

En el código cargaremos los datos relativos a las elecciones generales del 23 de julio de 2023 al congreso de los Diputados a nivel de mesa electoral, los datos relativos a las identificaciones de los partidos políticos y las geometrías de las secciones censales con el objetivo de configurar un fichero único.

En función de la elección del usuario se generarán distintos outputs, en primer lugar, se podrá determinar en función del tipo de archivo generado:

- GeoJson: formato estándar abierto diseñado para representar elementos geográficos sencillos, junto con sus atributos no espaciales, basado en JavaScript Object Notation.

- ShapeFile: formato ESRI Shapefile (SHP) es un formato de archivo informático propietario de datos espaciales desarrollado por la compañía ESRI, quien crea y comercializa software para Sistemas de Información Geográfica como Arc/Info o ArcGIS.

Por otro lado, el usuario puede determinar qué tipo de dato quiere generar:

- *votos_secciones*: este será el output generado que dispondrá de la información agregada por sección censal para todos los partidos, así como sus datos geométricos para la representación.
- *votos_secciones_simp*: mismo concepto que *votos_secciones* pero en este caso los partidos menos representativos* se agregan en la categoría *OTROS*.
- *votos_ganador*: Para determinadas visualizaciones se ha considerado interesante generar un fichero ligero que únicamente muestre el partido más votado por sección censal.




*Se consideran poco representativos y susceptibles de entrar en la categoría "OTROS" todos aquellos partidos que no superen un mínimo del 1,5% d votos a nivel provincial.



**Requerimientos**

Especificamos las librerías requeridas para la ejecución de los procesos:

- pandas >= 2.2.1
- geopandas >= 0.14.2

Versión de python utilizada: 3.10.13

In [1]:
# -*- coding: utf-8 -*-
import os
import shutil
import zipfile
import pandas as pd
import geopandas as gpd

**Extensión de los archivos geospaciales resultantes:**

In [77]:
save_as_geojson = True
save_as_shapefile = True


**Outputs a exportar**

In [76]:
save_alldata = False
save_simplified = False
save_winner = True

<a id='1'></a>

**Inputs necesarios**

- Datos de los resultados a nivel mesa electoral.
    - Fuente: [Ministerio del Interior](https://infoelectoral.interior.gob.es/es/elecciones-celebradas/area-de-descargas/)
- Geometrías de las secciones censales.
    - Fuente: [Instituto Nacional de Estadística](https://www.ine.es/ss/Satellite?L=es_ES&c=Page&cid=1259952026632&p=1259952026632&pagename=ProductosYServicios%2FPYSLayout)

Ambos conjuntos de datos estan incorporados dentro de la carpeta /data/inputs/ como: *02202307_MESA.zip* y *seccionado.zip*

In [4]:
# Establecemos la ruta a la raiz donde estaran los inputs y outputs
nb_path = os.path.join(os.getcwd(), 'data').replace('\\', '/')
# Definimos la ruta de outputs y creamos la carpeta de guardado
out_path = nb_path+'/outputs/'
os.makedirs(out_path, exist_ok=True)

### 1. Descomprimir inputs

In [5]:
# Función para descomprimir en una carpeta temporal
def unzip(zip, temp):
    with zipfile.ZipFile(zip, 'r') as archivo_zip:
        archivo_zip.extractall(path=temp)

#### 1.1 Descomprimir ficheros y cargar datos de los resultados de las elecciones generales

Al descomprimir se generan 12 archivos, de los cuales 2 son documentación y los 10 restantes ficheros .DAT que contienen la información a explotar (*03022307.DAT* contiene la información relativa a las nomenclaturas de los partidos políticos y *10022307.DAT* la información relativa a los resultados por partido y mesa electoral).

In [6]:
# Establecemos el nombre del fichero a descomprimir que contiene la información de las elecciones
zip_path = nb_path + '//inputs//02202307_MESA.zip'
# Nombramos una carpeta temporal donde descomprimir
temp_folder = r'data/inputs/temp'
#Llamamos a nuestra función de descompresión
unzip(zip_path,temp_folder)

In [7]:
# Una vez descomprimido, cargamos el archivo en un dataframe de pandas
parties_path = os.path.join(temp_folder, '03022307.DAT')
parties = pd.read_csv(parties_path, sep='\s{2,}', encoding='ISO-8859-1', header=None, dtype=str, engine='python')
parties.head(3)

,0,1,2
0,02202307000001FO,FRENTE OBRERO,000001000001000001
1,02202307000002PSOE,PARTIDO SOCIALISTA OBRERO ESPAÑOL,000002000002000002
2,02202307000003PUM+J,POR UN MUNDO MÁS JUSTO,000003000003000003


In [8]:
# Definimos la ruta del archivo y cargamos como dataframe
data_path = nb_path + '//inputs//temp//10022307.DAT'
data = pd.read_csv(data_path, sep=',', encoding='ISO-8859-1', header=None)
# Mostramos los primeros 3 registros del df
data.head(3)

,0
0,022023071010400101001 A0000010000001
1,022023071010400101001 A0000020000122
2,022023071010400101001 A0000030000000


#### 1.2 Descomprimir fichero y cargar geometrías de las secciones censales
Se descomprime una carpeta intermedia en la que encontraremos el archivo de extensión .shp asi como los archivos complementarios (.prj, .dbf, .sbn, etc.).

In [9]:
# Ruta del archivo zip y nombre del archivo dentro del zip que quieres leer
zip_path = nb_path + '//inputs//seccionado_2023.zip'
folder_zip = 'España_Seccionado2023_ETRS89H30/'
unzip(zip_path,temp_folder)

In [10]:
# En este caso al descomprimir hay una carpeta intermedia que debemos añadir a la path
shp_path = os.path.join(temp_folder, folder_zip, 'SECC_CE_20230101.shp')
# Cargamos los resultados en un geodataframe
census_gdf = gpd.read_file(shp_path)
census_gdf.head(3)

,CUSEC,CUMUN,CSEC,CDIS,CMUN,CPRO,CCA,CUDIS,CLAU2,NPRO,NCA,CNUT0,CNUT1,CNUT2,CNUT3,NMUN,geometry
0,0100101001,01001,001,01,001,01,16,0100101,01001,Araba/Álava,País Vasco,ES,2,1,1,Alegría-Dulantzi,"MULTIPOLYGON (((539753.044 4743324.668, 539784..."
1,0100101002,01001,002,01,001,01,16,0100101,01001,Araba/Álava,País Vasco,ES,2,1,1,Alegría-Dulantzi,"POLYGON ((539559.740 4745571.157, 539562.677 4..."
2,0100201001,01002,001,01,002,01,16,0100201,01002,Araba/Álava,País Vasco,ES,2,1,1,Amurrio,"MULTIPOLYGON (((503618.553 4759559.798, 503620..."


In [11]:
# Una vez cargados todos los datos, procedemos a eliminar la carpeta temporal
shutil.rmtree(temp_folder)

**Definimos funciones que serán utilizadas en el formateo y normalización de los datos**

In [12]:
# Creamos una función para formatear el dataframe de partidos políticos
def clean_parties(df):
    df[2] = df[2].astype('str')
    df['id_partido_provincial'] = df[2].str[:6]
    df['id_partido_autonomico'] = df[2].str[6:12]
    df['id_partido_nacional'] = df[2].str[-6:]
    df['partido'] = df[1]
    df = df.drop([0,1,2], axis=1)
    df = df.drop_duplicates()
    return df

In [13]:
# Renombramos los partidos para unificar terminología independientemente de su id_partido_xx
def replacenames_duplicate_parties(df):
    df.loc[df['id_partido_nacional'] == '000010', 'partido'] = 'SUMAR'
    df.loc[df['id_partido_nacional'] == '000002', 'partido'] = 'PSOE'
    df.loc[df['id_partido_nacional'] == '000015', 'partido'] = 'PCTE'
    df.loc[df['id_partido_nacional'] == '000003', 'partido'] = 'POR UN MUNDO MÁS JUSTO'
    df.loc[df['id_partido_nacional'] == '000005', 'partido'] = 'PP'
    df.loc[df['id_partido_nacional'] == '000007', 'partido'] = 'PACMA'
    df.loc[df['id_partido_nacional'] == '000014', 'partido'] = 'ESCAÑOS EN BLANCO PARA DEJAR ESCAÑOS VACÍOS'
    df.loc[df['id_partido_nacional'] == '000023', 'partido'] = 'COALICIÓN EXISTE'
    df.loc[df['id_partido_nacional'] == '000075', 'partido'] = 'PNV'

    df.loc[df['id_partido_autonomico'] == '000030', 'partido'] = 'SUMAR'
    df.loc[df['id_partido_autonomico'] == '000033', 'partido'] = 'SUMAR'
    df.loc[df['id_partido_autonomico'] == '000066', 'partido'] = 'SUMAR'
    df.loc[df['id_partido_autonomico'] == '000021', 'partido'] = 'SUMAR'
    df.loc[df['id_partido_autonomico'] == '000011', 'partido'] = 'SUMAR'
    df.loc[df['id_partido_autonomico'] == '000051', 'partido'] = 'SUMAR'
    df.loc[df['id_partido_autonomico'] == '000067', 'partido'] = 'POR UN MUNDO MÁS JUSTO'

    df.loc[df['id_partido_autonomico'] == '000072', 'partido'] = 'PSOE'
    df.loc[df['id_partido_autonomico'] == '000029', 'partido'] = 'PSOE'
    df.loc[df['id_partido_autonomico'] == '000076', 'partido'] = 'PSOE'
    df.loc[df['id_partido_autonomico'] == '000056', 'partido'] = 'PP'
    df.loc[df['id_partido_autonomico'] == '000064', 'partido'] = 'PSOE'
    df.loc[df['id_partido_autonomico'] == '000068', 'partido'] = 'PCTE'
    df.loc[df['id_partido_autonomico'] == '000078', 'partido'] = 'PCTE'
    df.loc[df['id_partido_autonomico'] == '000068', 'partido'] = 'PCTE'
    df.loc[df['id_partido_autonomico'] == '000082', 'partido'] = 'SUMAR'

    df = df.drop_duplicates()
    return df

In [14]:
# Unificamos los id provinciales, autonómicos y nacionales en un único id.
def get_single_id(df):
    df_prov = df[['id_partido_provincial','partido']]
    df_prov = df_prov.rename(columns={'id_partido_provincial': 'id_partido'})

    df_auto = df[['id_partido_autonomico','partido']]
    df_auto = df_auto.rename(columns={'id_partido_autonomico': 'id_partido'})

    df_nac = df[['id_partido_nacional','partido']]
    df_nac = df_nac.rename(columns={'id_partido_nacional': 'id_partido'})

    df_concat = pd.concat([df_prov, df_auto, df_nac])
    df_concat = df_concat.drop_duplicates()

    return df_concat

In [15]:
# Función para convertir un dataframe en un geodataframe
def df2gdf(df, geometry_field, crs_type):
    gdf = gpd.GeoDataFrame(df, geometry=geometry_field, crs=crs_type)
    return gdf

### 2. Formateo partidos politicos df

**Nota técnica: Partidos políticos**

Tras separar los bloques por 2 o más espacios encontramos en la columna 2 los id relativos a la candidatura.

Son 3 códigos: código de la candidatura cabecera de acumulación a nivel provincial (posiciones ), a nivel autonómico y a nivel nacional.

Más info en el documento *FICHEROS.doc* que viene en el zip.

In [16]:
# Formateamos el df de partidos políticos
parties = clean_parties(parties)
parties.head(3)

,id_partido_provincial,id_partido_autonomico,id_partido_nacional,partido
0,000001,000001,000001,FRENTE OBRERO
1,000002,000002,000002,PARTIDO SOCIALISTA OBRERO ESPAÑOL
2,000003,000003,000003,POR UN MUNDO MÁS JUSTO


In [17]:
# Renombramos los partidos politicos que se presentan con diferente nombre en distintos territorios
parties = replacenames_duplicate_parties(parties)
parties.head(3)

,id_partido_provincial,id_partido_autonomico,id_partido_nacional,partido
0,000001,000001,000001,FRENTE OBRERO
1,000002,000002,000002,PSOE
2,000003,000003,000003,POR UN MUNDO MÁS JUSTO


In [18]:
# Unificamos los id de partido en uno único
parties = get_single_id(parties)
parties.head(3)

,id_partido,partido
0,000001,FRENTE OBRERO
1,000002,PSOE
2,000003,POR UN MUNDO MÁS JUSTO


### 3. Formateo resultados electorales df
Se obtendrán a partir del código del fichero los siguientes campos:
- Comunidad Autónoma
- Provincia
- Municipio
- Distrito y sección censal
- Votos

  
Todos los valores obtenidos para los campos quedan definidos en el documento *FICHEROS.doc*

In [19]:
data.head(3)

,0
0,022023071010400101001 A0000010000001
1,022023071010400101001 A0000020000122
2,022023071010400101001 A0000030000000


In [20]:
# Mostramos la extensión del dataframe
data.shape

(647309, 1)

#### 3.1 Comunidad Autónoma

In [21]:
# Función para obtener la comunidad autónoma como id
def get_region(df):
    df['comunidad_autonoma'] = df[0].str[9:11]
    return df

In [22]:
# Obtenemos la CCAA
data = get_region(data)
data.head(2)

,0,comunidad_autonoma
0,022023071010400101001 A0000010000001,01
1,022023071010400101001 A0000020000122,01


#### 3.2 Provincia

In [23]:
# Función para obtener la provincia como id y añadir el nombre
def get_province(df):
    df['provincia'] = df[0].str[11:13]
    prov_data = {'01': 'Araba/Álava', '02': 'Albacete', '03': 'Alicante/Alacant', '04': 'Almería',
                 '05': 'Ávila', '06': 'Badajoz', '07': 'Balears, Illes', '08': 'Barcelona', '09': 'Burgos',
                 '10': 'Cáceres', '11': 'Cádiz', '12': 'Castellón/Castelló', '13': 'Ciudad Real', '14': 'Córdoba',
                 '15': 'Coruña, A', '16': 'Cuenca', '17': 'Girona', '18': 'Granada', '19': 'Guadalajara', '20': 'Gipuzkoa',
                 '21': 'Huelva', '22': 'Huesca', '23': 'Jaén', '24': 'León', '25': 'Lleida', '26': 'Rioja, La', '27': 'Lugo',
                 '28': 'Madrid', '29': 'Málaga', '30': 'Murcia', '31': 'Navarra', '32': 'Ourense', '33': 'Asturias', '34': 'Palencia',
                 '35': 'Palmas, Las', '36': 'Pontevedra', '37': 'Salamanca', '38': 'Santa Cruz de Tenerife', '39': 'Cantabria', '40': 'Segovia',
                 '41': 'Sevilla', '42': 'Soria', '43': 'Tarragona', '44': 'Teruel', '45': 'Toledo', '46': 'Valencia/València', '47': 'Valladolid',
                 '48': 'Bizkaia', '49': 'Zamora', '50': 'Zaragoza', '51': 'Ceuta', '52': 'Melilla'}
    prov = pd.DataFrame(list(prov_data.items()), columns=['id_prov', 'provincia_nom'])
    df = pd.merge(df, prov, left_on='provincia', right_on='id_prov', how='left')
    df = df.drop(['id_prov'], axis=1)
    return df

In [24]:
# Obtenemos la provincia
data = get_province(data)
data.head(2)

,0,comunidad_autonoma,provincia,provincia_nom
0,022023071010400101001 A0000010000001,01,04,Almería
1,022023071010400101001 A0000020000122,01,04,Almería


#### 3.3 Municipio

In [25]:
# Función para obtener el id del municipio (id con formato INE: id_prov + id_muni)
def get_municipality(df):
    df['municipio'] = data[0].str[11:16]
    return df

In [26]:
data = get_municipality(data)
data.head(2)

,0,comunidad_autonoma,provincia,provincia_nom,municipio
0,022023071010400101001 A0000010000001,01,04,Almería,04001
1,022023071010400101001 A0000020000122,01,04,Almería,04001


#### 3.4 Distrito y Sección Censal

In [27]:
# Función para obtener la sección censal (la seccion censal está compuesta por el distrito + sección censal, para formatear con el tipo de la geometría añadimos el municipio)
def get_census_section(df):
    df['seccion_censal'] = df[0].str[16:21]
    df['seccion_censal'] = df['municipio'] + df['seccion_censal']
    return df

In [28]:
# Obtenemos el distrito y sección censal
data = get_census_section(data)
data.head(3)

,0,comunidad_autonoma,provincia,provincia_nom,municipio,seccion_censal
0,022023071010400101001 A0000010000001,01,04,Almería,04001,0400101001
1,022023071010400101001 A0000020000122,01,04,Almería,04001,0400101001
2,022023071010400101001 A0000030000000,01,04,Almería,04001,0400101001


#### 3.5 Partido político

In [29]:
# Función para obtener el id del partido político
def get_id_partie(df):
    df['id_partido'] = df[0].str[23:29]
    return df

In [30]:
# Obtenemos el id de partido político, con este id podremos cruzar con la tabla anterior donde tenemos el nombre del partido
data = get_id_partie(data)
data.head(3)

,0,comunidad_autonoma,provincia,provincia_nom,municipio,seccion_censal,id_partido
0,022023071010400101001 A0000010000001,01,04,Almería,04001,0400101001,000001
1,022023071010400101001 A0000020000122,01,04,Almería,04001,0400101001,000002
2,022023071010400101001 A0000030000000,01,04,Almería,04001,0400101001,000003


#### 3.6 Votos obtenidos

In [31]:
# Función para retornar el valor de votos recibidos y lo casteamos como entero
def get_votes(df):
    df['votos'] = df[0].str[29:36]
    df['votos'] = df['votos'].astype('int')
    return df

In [32]:
# Obtenemos los votos
data = get_votes(data)
data.head(3)

,0,comunidad_autonoma,provincia,provincia_nom,municipio,seccion_censal,id_partido,votos
0,022023071010400101001 A0000010000001,01,04,Almería,04001,0400101001,000001,1
1,022023071010400101001 A0000020000122,01,04,Almería,04001,0400101001,000002,122
2,022023071010400101001 A0000030000000,01,04,Almería,04001,0400101001,000003,0


#### 3.7 Filtros

Dentro del conjunto de los datos hay agregados para los votos CERA* (Censo Electoral de Residentes Ausentes) a nivel provincial y autonómico. Han de filtrarse ya que están identificados individualmente a nivel de mesa electoral.

Para nuestra representación cartografica no tiene impacto ya que solamente nos quedaremos con aquellos datos que dispongan de una geometría asociada, pero consideramos importante tener en cuenta este proceso de limpieza para futuros procesos con estos mismos datos.

(*) Son los votos de aquellas personas que viven permanentemente fuera de España y que están inscritos en la Oficina Consular. Estos ciudadanos pueden tomar parte tanto de las elecciones generales como de las elecciones al Parlamento Europeo y a la comunidad autónoma de la que provengan.

In [33]:
data.shape

(647309, 8)

In [34]:
# Excluimos aquellos datos cuya comunidad auntónoma o provincia sea igual a 99
data = data[(data['comunidad_autonoma']!='99')]
data = data[(data['provincia']!='99')]

In [35]:
data.shape

(647022, 8)

### 4. Agregado de las mesas

A continuación agregaremos todas las filas del df con el objetivo de tener el valor de votos por partido a nivel de sección censal.

In [36]:
# Agrergamos usando groupby
data_secc = data.groupby(['comunidad_autonoma','provincia', 'provincia_nom', 'municipio', 'seccion_censal','id_partido'])['votos'].sum().reset_index(name='votos')
data_secc.head(5)

,comunidad_autonoma,provincia,provincia_nom,municipio,seccion_censal,id_partido,votos
0,01,04,Almería,04001,0400101001,000001,4
1,01,04,Almería,04001,0400101001,000002,294
2,01,04,Almería,04001,0400101001,000003,0
3,01,04,Almería,04001,0400101001,000004,6
4,01,04,Almería,04001,0400101001,000005,262


### 5. Unión con partidos

Unimos el df que acabamos de agrupar con la tabla de partidos para disponer del nombre de cada partido

In [37]:
# Interesectamos nuestra tambla principal con la tabla de partidos aplicando un left join
data_secc = data_secc.merge(parties, on='id_partido', how='left')
data_secc.head(3)

,comunidad_autonoma,provincia,provincia_nom,municipio,seccion_censal,id_partido,votos,partido
0,01,04,Almería,04001,0400101001,000001,4,FRENTE OBRERO
1,01,04,Almería,04001,0400101001,000002,294,PSOE
2,01,04,Almería,04001,0400101001,000003,0,POR UN MUNDO MÁS JUSTO


In [38]:
# Chequeamos los nombres de partidos que se han unido al df
data_secc['partido'].unique()

array(['FRENTE OBRERO', 'PSOE', 'POR UN MUNDO MÁS JUSTO',
       'ALMERIENSES - REGIONALISTAS PRO ALMERÍA', 'PP', 'VOX', 'PACMA',
       'LIBRES', 'RECORTES CERO', 'SUMAR', 'ADELANTE ANDALUCÍA',
       'CAMINANDO JUNTOS', 'ESCAÑOS EN BLANCO PARA DEJAR ESCAÑOS VACÍOS',
       'PCTE', 'JUNTOS POR GRANADA', 'POR HUELVA', 'JAÉN MERECE MÁS',
       'FALANGE ESPAÑOLA DE LAS J.O.N.S.',
       'FEDERACIÓN DE LOS INDEPENDIENTES DE ARAGÓN', 'COALICIÓN EXISTE',
       'PARTIDO ARAGONÉS', 'PARTIDO UNIONISTA ESTADO DE ESPAÑA',
       'ASTURIAS EXISTE-ESPAÑA VACIADA', 'COALICIÓN CANARIA',
       'NUEVA CANARIAS - BLOQUE CANARISTA',
       'AHORA CANARIAS-PARTIDO COMUNISTA DEL PUEBLO CANARIO', 'POR ÁVILA',
       'VÍA BURGALESA',
       'ESPAÑA VACIADA-PARTIDO CASTELLANO-TIERRA COMUNERA',
       'ESPAÑA VACIADA', 'PARTIDO REGIONALISTA DEL PAÍS LEONÉS',
       'UNIÓN DEL PUEBLO LEONÉS', 'VAMOS PALENCIA',
       'GRUPO INDEPENDIENTE PALENCIA TIERRA VIVA',
       'TERCERA EDAD EN ACCIÓN', 'SORIA ¡YA!', 

### 6. Obtención de los partidos representativos

El objetivo de este procedimiento es la simplificación de partidos para poder generar un fichero más ligero y sencillo de trabajar. Este fichero será el que hemos denominado *votos_secciones_simp*

Lo que vamos a hacer a continuación es quedarnos únicamente con los partidos que al menos lleguen a un porcentaje minimo de votos, quedando todos los que no cumplan dicha condición en un agregado denominado *OTROS*.

Para ello:
1. Calculamos los votos por partido a nivel provincial.
2. Calculamos el porcentaje que representan dichos votos.
3. Obtenemos un listado de partidos que superen por provincia un limite de representatividad (1,5%).
4. Creamos un nuevo campo 'partido_simp' que aplica la categoría 'OTROS' a aquellos partidos que no superen el límite establecido.

#### 6.1 Agregado a nivel provincia

In [39]:
# Agregamos los votos obtenidos por partido y provincia
partidos_provincia = data_secc.groupby(['provincia','partido'])['votos'].sum().reset_index().sort_values(by='provincia', ascending=True)
partidos_provincia.head(2)

,provincia,partido,votos
0,01,ESCAÑOS EN BLANCO PARA DEJAR ESCAÑOS VACÍOS,619
1,01,EUSKAL HERRIA BILDU,32987


#### 6.2 Porcentaje de los votos por partido

In [40]:
# Definimos una función para obtener el porcentaje de votos por provincia 
def get_percentaje_vote_prov(df):
    df_totales = df.groupby(['comunidad_autonoma','provincia'])['votos'].sum().reset_index(name='votos').sort_values(by='votos', ascending=False)
    df_totales = df_totales.rename(columns={'votos': 'votos_totales'})
    df_join = pd.merge(df, df_totales, on=['comunidad_autonoma','provincia'], how='left')
    df_join['porcentaje_voto_prov'] = df_join['votos']/df_join['votos_totales']
    df_join = df_join.groupby(['comunidad_autonoma','provincia','partido'])['porcentaje_voto_prov'].sum().reset_index().sort_values(by='porcentaje_voto_prov', ascending=False)
    return df_join

In [41]:
# Aplicamos la función para obtener el porcentaje
representative_parties = get_percentaje_vote_prov(data_secc)
representative_parties.head(3)

,comunidad_autonoma,provincia,partido,porcentaje_voto_prov
400,11,27,PP,0.505633
411,11,32,PP,0.503246
545,19,52,PP,0.496236


#### 6.3 Obtención de la lista de partidos que superen el límite establecido

In [42]:
# Definimos un limite porcentual de votos, consideramos un mínimo de un 1,5% de los votos
limit = 0.015
# Definimos una función que filtre el df por el limite establecido y simplifique los partidos obtenidos en una lista 
def get_representative_parties(df, limit):
    df = df.groupby(['comunidad_autonoma','provincia','partido'])['porcentaje_voto_prov'].sum().reset_index().sort_values(by='porcentaje_voto_prov', ascending=False)
    df = df[df['porcentaje_voto_prov']>=limit]
    df = df.groupby(['partido']).sum().reset_index().sort_values(by='porcentaje_voto_prov', ascending=False)
    list_parties = df['partido'].tolist()
    return list_parties

In [43]:
# Aplicamos la función y obtenemos los partidos
representative_parties_list = get_representative_parties(representative_parties, limit)
representative_parties_list

['PP',
 'PSOE',
 'VOX',
 'SUMAR',
 'EUSKAL HERRIA BILDU',
 'PNV',
 'ESQUERRA REPUBLICANA DE CATALUNYA',
 'JUNTS PER CATALUNYA - JUNTS',
 'BLOQUE NACIONALISTA GALEGO',
 'COALICIÓN CANARIA',
 'SORIA ¡YA!',
 'COALICIÓN EXISTE',
 'UNION DEL PUEBLO NAVARRO',
 "CANDIDATURA D'UNITAT POPULAR-PER LA RUPTURA",
 'UNIÓN DEL PUEBLO LEONÉS',
 'NUEVA CANARIAS - BLOQUE CANARISTA',
 'POR ÁVILA',
 'COALICIÓN POR MELILLA',
 'GEROA BAI',
 'JAÉN MERECE MÁS',
 'VAMOS PALENCIA',
 'ZAMORA SÍ']

#### 6.4 Identificamos los partidos representativos

Retornamos a nuestro dataframe de votos agregados a nivel de sección censal y aplicamos un mapeo de los partidos no representativos.

In [44]:
# Definimos una función que aplique el término 'OTROS' a aquellos items no contenidos en la lista, se aplicara dentro de una función lambda para cada fila del df
def check_partie(partie):
    if partie not in representative_parties_list:
        return 'OTROS'
    else:
        return partie

In [45]:
# Creamos una nueva columna que aplique la función anteriormente definida mediante una función lambda
data_secc['partido_simp'] = data_secc['partido'].apply(lambda x: check_partie(x))
data_secc.head(3)

,comunidad_autonoma,provincia,provincia_nom,municipio,seccion_censal,id_partido,votos,partido,partido_simp
0,01,04,Almería,04001,0400101001,000001,4,FRENTE OBRERO,OTROS
1,01,04,Almería,04001,0400101001,000002,294,PSOE,PSOE
2,01,04,Almería,04001,0400101001,000003,0,POR UN MUNDO MÁS JUSTO,OTROS


### 7. 	Porcentaje de voto a nivel de sección censal

In [46]:
# Construimos una función para obtener el porcentaje de voto a nivel de sección censal. Retornará un nuevo campo llamado 'porcentaje_voto_secc'
def get_percentaje_vote_censussection(df):
    df_totales = df.groupby(['comunidad_autonoma','provincia','municipio','seccion_censal'])['votos'].sum().reset_index(name='votos').sort_values(by='votos', ascending=False)
    df_totales = df_totales.rename(columns={'votos': 'votos_totales'})
    df_join = pd.merge(df, df_totales, on=['comunidad_autonoma','provincia','municipio','seccion_censal'], how='left')
    df_join['porcentaje_voto_secc'] = df_join['votos']/df_join['votos_totales'].round(3)
    return df_join

In [47]:
out_path

'c:/Users/lopez/Documents/GitHub/spanish_elections_results/data/outputs/'

In [48]:
# Implementamos la función en el df
data_census = get_percentaje_vote_censussection(data_secc)
data_census.head(3)

,comunidad_autonoma,provincia,provincia_nom,municipio,seccion_censal,id_partido,votos,partido,partido_simp,votos_totales,porcentaje_voto_secc
0,01,04,Almería,04001,0400101001,000001,4,FRENTE OBRERO,OTROS,737,0.005427
1,01,04,Almería,04001,0400101001,000002,294,PSOE,PSOE,737,0.398915
2,01,04,Almería,04001,0400101001,000003,0,POR UN MUNDO MÁS JUSTO,OTROS,737,0.000000


### 8. Unimos las geometrías al dataframe

En este paso primero aplicaremos una serie de formateos al geodataframe, generaremos un subconjunto formado únicamente por los campos que nos interesen y finalmente haremos un left join con el df principal asociando a cada sección censal su geometría.

#### 8.1 Formateo del geodataframe de secciones censales

In [49]:
# Función para construir un campo geo_seccion_censal el id de las secciones censales a partir de la concatenación del código municipio, distrito y sección censal
def get_geo_census_section(gdf):
    gdf['geo_seccion_censal'] = gdf['CUMUN'] + gdf['CDIS'] + gdf['CSEC']
    return gdf

In [50]:
# Aplicamos la función anteriormente definida
census_gdf = get_geo_census_section(census_gdf)
census_gdf.head(3)

,CUSEC,CUMUN,CSEC,CDIS,CMUN,CPRO,CCA,CUDIS,CLAU2,NPRO,NCA,CNUT0,CNUT1,CNUT2,CNUT3,NMUN,geometry,geo_seccion_censal
0,0100101001,01001,001,01,001,01,16,0100101,01001,Araba/Álava,País Vasco,ES,2,1,1,Alegría-Dulantzi,"MULTIPOLYGON (((539753.044 4743324.668, 539784...",0100101001
1,0100101002,01001,002,01,001,01,16,0100101,01001,Araba/Álava,País Vasco,ES,2,1,1,Alegría-Dulantzi,"POLYGON ((539559.740 4745571.157, 539562.677 4...",0100101002
2,0100201001,01002,001,01,002,01,16,0100201,01002,Araba/Álava,País Vasco,ES,2,1,1,Amurrio,"MULTIPOLYGON (((503618.553 4759559.798, 503620...",0100201001


#### 8.2 Generamos el subconjunto que uniremos

In [51]:
# Definimos una función para renombrar los campos del geodataframe que nos van a servir más adelante y retornamos un subconjunto del original 
def clean_geo_census_section(gdf):
    gdf = gdf.rename(columns={'NCA': 'geo_comunidad_autonoma_nom','NMUN': 'geo_municipio_nom'})
    gdf = gdf[['geo_seccion_censal', 'geo_municipio_nom', 'geo_comunidad_autonoma_nom', 'geometry']]
    return gdf

In [52]:
# Lo aplicamos
secc_gdf = clean_geo_census_section(census_gdf)
secc_gdf.head(3)

,geo_seccion_censal,geo_municipio_nom,geo_comunidad_autonoma_nom,geometry
0,0100101001,Alegría-Dulantzi,País Vasco,"MULTIPOLYGON (((539753.044 4743324.668, 539784..."
1,0100101002,Alegría-Dulantzi,País Vasco,"POLYGON ((539559.740 4745571.157, 539562.677 4..."
2,0100201001,Amurrio,País Vasco,"MULTIPOLYGON (((503618.553 4759559.798, 503620..."


#### 8.3 Unión con los datos de votos a nivel sección censal

In [61]:
# Nuevo df resultado de la unión y con la información de la geometría
all_parties_census = pd.merge(data_census, secc_gdf, left_on='seccion_censal', right_on='geo_seccion_censal', how='left')
all_parties_census.head(3)

#### 9 Chequeo unión final y filtrado datos sin geometría

In [63]:
# Chequeamos el número de secciones censales con datos de voto pero sin geometría asociada:
print('Número secciones censales con datos de voto pero sin geometría asociada: ' +
      str(all_parties_census[all_parties_census['geometry']==None].shape[0])+' - '+
      str(
          round((all_parties_census[all_parties_census['geometry']==None].shape[0])/(all_parties_census.shape[0]),1))+
      '%'
     )

Número secciones censales con datos de voto pero sin geometría asociada: 1032 - 0.0%


In [64]:
# Chequeo de secciones censales con geometría pero sin datos de votos:
check_geoms_nodata = pd.merge(data_census, census_gdf, left_on='seccion_censal', right_on='geo_seccion_censal', how='outer')
print('Geometrias de secciones censales que no tienen con datos de voto: ' +
      str(check_geoms_nodata[check_geoms_nodata['partido'].isnull()].shape[0])+' - '+
      str(round((check_geoms_nodata[check_geoms_nodata['partido'].isnull()].shape[0])/(check_geoms_nodata.shape[0]),1))+'%'
      )

Geometrias de secciones censales que no tienen con datos de voto: 49 - 0.0%


In [65]:
# Filtro de datos sin geometría
all_parties_census = all_parties_census[all_parties_census['geometry']!=None]
print('Items válidos a exportar: ' + str(all_parties_census.shape[0]))

Items válidos a exportar: 392197


### 10 Generación y guardado de los outputs finales

Guardado de los ficheros finales:
- Convertimos en geodataframe
- Guardado condicional como GeoJson (votos_secciones.geojson)
- Guardado condicional como Shapefile (votos_secciones.shp)
- Agregado partidos simplificados
- Guardado condicional como GeoJson (votos_secciones_simp.geojson)
- Guardado condicional como Shapefile (votos_secciones_simp.shp)

#### 10.1 Convertimos en geodataframe

In [66]:
# Cambiamos de dataframe a geodataframe
all_parties_census = df2gdf(all_parties_census, 'geometry', 'EPSG:25830')
type(all_parties_census)

geopandas.geodataframe.GeoDataFrame

##### 10.1.1 Guardado condicional como GeoJson (votos_secciones.geojson)

In [67]:
# Guardar el GeoDataFrame como GeoJSON
if save_as_geojson and save_alldata:
    output_file = out_path + 'votos_secciones.geojson'
    all_parties_census.to_file(output_file, driver='GeoJSON')

#### 10.1.2 Guardado condicional como GeoJson (votos_secciones.shp)

In [68]:
if save_as_shapefile and save_alldata:
    output_file = out_path + 'votos_secciones.shp'
    all_parties_census.to_file(output_file, driver="ESRI Shapefile")

#### 10.2 Agregado partidos representativos

In [70]:
# Agregamos para sumar votos y porcentajes de los partidos simplificados ('OTROS')
all_parties_simp_census = all_parties_census.groupby(['comunidad_autonoma','provincia', 'provincia_nom','municipio','seccion_censal',
                                                  'partido_simp'])[['votos','porcentaje_voto_secc']].sum() \
                                                    .reset_index().sort_values(by='votos', ascending=False)
# Unimos con el gdf de geometrías
all_parties_simp_census = pd.merge(all_parties_simp_census, secc_gdf, left_on='seccion_censal', right_on='geo_seccion_censal', how='left')
# Convertimos a gdf
all_parties_simp_census = df2gdf(all_parties_simp_census, 'geometry', 'EPSG:25830')
all_parties_simp_census.head(3)

,comunidad_autonoma,provincia,provincia_nom,municipio,seccion_censal,partido_simp,votos,porcentaje_voto_secc,geo_seccion_censal,geo_municipio_nom,geo_comunidad_autonoma_nom,geometry
0,12,28,Madrid,28006,2800601059,PP,1205,0.759294,2800601059,Alcobendas,Comunidad de Madrid,"POLYGON ((446242.970 4484314.788, 445987.627 4..."
1,12,28,Madrid,28009,2800901001,PP,1171,0.680814,2800901001,Algete,Comunidad de Madrid,"POLYGON ((452723.770 4500115.242, 452613.564 4..."
2,12,28,Madrid,28022,2802201025,PP,1149,0.645506,2802201025,Boadilla del Monte,Comunidad de Madrid,"POLYGON ((424278.600 4475212.160, 424276.034 4..."


#### 10.2.1 Guardado condicional como GeoJson (votos_secciones_simp.geojson)

In [ ]:
# Guardar el GeoDataFrame como GeoJSON
if save_as_geojson and save_simplified:
    output_file = out_path + 'votos_secciones_simp.geojson'
    all_parties_simp_census.to_file(output_file, driver='GeoJSON')

In [ ]:
if save_as_shapefile and save_simplified:
    output_file = out_path + 'votos_secciones_simp.shp'
    all_parties_simp_census.to_file(output_file, driver="ESRI Shapefile")

#### 10.3 Filtrado partido más votado

In [85]:
win_partie_census.head(3)

,comunidad_autonoma,provincia,provincia_nom,municipio,seccion_censal,partido_simp,votos,porcentaje_voto_secc,geo_seccion_censal,geo_municipio_nom,geo_comunidad_autonoma_nom,geometry
173112,08,42,Soria,42051,4205101001,PP,11,1.0,4205101001,Carabantes,Castilla y León,"POLYGON ((586128.571 4599893.258, 585854.569 4..."
207254,08,09,Burgos,09184,0918401001,VOX,1,1.0,0918401001,Jaramillo Quemado,Castilla y León,"POLYGON ((473676.338 4660168.347, 473262.332 4..."
118305,14,20,Gipuzkoa,20060,2006001001,EUSKAL HERRIA BILDU,62,1.0,2006001001,Orexa,País Vasco,"POLYGON ((581558.582 4772554.186, 581572.332 4..."


In [83]:
win_partie_census = all_parties_simp_census.sort_values(by=['porcentaje_voto_secc'], ascending=False)
win_partie_census = win_partie_census.drop_duplicates(subset=['comunidad_autonoma', 'provincia', 'provincia_nom', 'municipio', 'seccion_censal',
                                           'geo_seccion_censal', 'geo_municipio_nom', 'geo_comunidad_autonoma_nom', 'geometry'])
win_partie_census.head(3)

,comunidad_autonoma,provincia,provincia_nom,municipio,seccion_censal,partido_simp,votos,porcentaje_voto_secc,geo_seccion_censal,geo_municipio_nom,geo_comunidad_autonoma_nom,geometry
173112,08,42,Soria,42051,4205101001,PP,11,1.0,4205101001,Carabantes,Castilla y León,"POLYGON ((586128.571 4599893.258, 585854.569 4..."
207254,08,09,Burgos,09184,0918401001,VOX,1,1.0,0918401001,Jaramillo Quemado,Castilla y León,"POLYGON ((473676.338 4660168.347, 473262.332 4..."
118305,14,20,Gipuzkoa,20060,2006001001,EUSKAL HERRIA BILDU,62,1.0,2006001001,Orexa,País Vasco,"POLYGON ((581558.582 4772554.186, 581572.332 4..."


In [87]:
win_partie_census[win_partie_census['geo_seccion_censal']=='2006001001']

,comunidad_autonoma,provincia,provincia_nom,municipio,seccion_censal,partido_simp,votos,porcentaje_voto_secc,geo_seccion_censal,geo_municipio_nom,geo_comunidad_autonoma_nom,geometry
118305,14,20,Gipuzkoa,20060,2006001001,EUSKAL HERRIA BILDU,62,1.0,2006001001,Orexa,País Vasco,"POLYGON ((581558.582 4772554.186, 581572.332 4..."


In [ ]:
# Guardar el GeoDataFrame como GeoJSON
if save_as_geojson and save_winner:
    output_file = out_path + 'votos_ganador.geojson'
    win_partie_census.to_file(output_file, driver='GeoJSON')

In [88]:
if save_as_shapefile and save_winner:
    output_file = out_path + 'votos_ganador.shp'
    win_partie_census.to_file(output_file, driver="ESRI Shapefile")

C:\Users\lopez\AppData\Local\Temp\ipykernel_24376\1510332767.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  win_partie_census.to_file(output_file, driver="ESRI Shapefile")
